In [ ]:
import os
import sys

project_root = os.getcwd()
while "src" not in os.listdir(project_root):
    project_root = os.path.dirname(project_root)
sys.path.append(project_root)

In [ ]:
import torch
from src.models import SimpleUNet, SCUNet, DeepSampler

In [ ]:
model = DeepSampler()
x = torch.randn(1, 1, 1025, 173)  # (batch, channels, height, width)
output = model(x)  # Salida: (1, 4, 1025, 173)
output.shape

In [ ]:
model = SimpleUNet()
x = torch.randn(1, 1, 1025, 173)  # (batch, channels, height, width)
output = model(x)  # Salida: (1, 4, 1025, 173)
output.shape

In [ ]:
model = SCUNet()
x = torch.randn(1, 1, 1025, 173)  # (batch, channels, height, width)
output = model(x)  # Salida: (1, 4, 1025, 173)
output.shape